# 1.Project Overview & Data Acquisition

## 1.1 Introduction

This project utilizes raw data sourced from the public Australian websites of three major supermarket chains: Woolworths, Aldi, and Coles. The datasets primarily include essential product attributes such as **SKU name, category, and price information**.

## 1.2 Data Acquisition Method

Product information was extracted from publicly accessible web pages using an automated Python script, thereby establishing a pricing monitoring dataset.

## 1.3 Compliance and Usage Declaration

**This project strictly adheres to ethical data collection standards.**

* All data was collected exclusively from publicly accessible webpages.
* The collection process did not involve logging in, bypassing access restrictions, or accessing non-public data.
* The datasets are used solely for the purpose of **technical learning and skill development**, and do not involve commercial use or large-scale redistribution.

# 2. Raw Data Description & Schema

The following table summarizes the key characteristics of the raw data collected from the three supermarket websites.

| Supermarket | Acquisition Date | Table Name | Rows | Columns |
| :--- | :--- | :--- | :--- | :--- |
| Woolworths | 2025-11-11 | wws_drinks | 2687 | 11 |
| Aldi | 2025-11-11 | aldi_drinks | 177 | 7 |
| Coles | 2025-11-10 | coles_drinks | 1485 | 8 |

---

## 2.1 Column Definitions

| Column Name | Description |
| :--- | :--- |
| **name** | Display name of the beverage product. |
| **brand** | Brand name of the beverage. |
| **instore_price** | Current in-store price (Lower than `instore_was_price` if on promotion). |
| **instore_was_price** | Original in-store price. |
| **instore_cup_price** | In-store price per `cup_measure` unit. |
| **online_price** | Current online price. |
| **online_was_price** | Original online price. |
| **online_cup_price** | Online price per `cup_measure` unit. |
| **package_size** | Packaging volume/capacity of the product. |
| **save_percent** | Discount percentage. |
| **category** | Product classification. |

---

## 2.2 Data Collection Nuances & Missingness Notes

**Important Context for Data Missingness:**

* **Aldi:** The official Aldi website only displays products and does not operate an online store. Consequently, the Aldi dataset contains **only in-store pricing data** and has no online price columns.
* **Coles:** The Coles online page exclusively displays online pricing. As a result, the Coles dataset has **no in-store pricing data** (e.g., `instore_price`, `instore_was_price`).

# 3.Data Cleaning

## 3.1 Initial Data Cleaning and Pre-Processing (Excel Operations)

The first phase of data cleaning was conducted using Microsoft Excel's functionalities for quick structural adjustments, noise removal, and initial missing value imputation.

### 3.1.1 Duplicate Removal

Duplicates were removed across all three datasets to ensure unique records.

| Table Name | Rows Removed | Remaining Rows | Tool Used |
| :--- | :--- | :--- | :--- |
| `wws_drinks` | 790 | 1896 | Excel (Remove Duplicates) |
| `aldi_drinks` | 0 | 177 | Excel (Remove Duplicates) |
| `coles_drinks` | 1 | 1484 | Excel (Remove Duplicates) |

### 3.1.2 Non-Core Product and Outlier Removal

Non-beverage products, promotional hampers, and extreme outliers were identified and removed.

#### Woolworths (`wws_drinks`)

* **Handling Logic:** Filters were applied to the `category` column to identify and remove gift/hampers.
* **Categories Removed:**
    * `["Hampers & Gifting", "Pantry"]`: 16 rows
    * `["Gifts, Wrapping & Cards", "Hampers & Gifting", "Pantry"]`: 14 rows
* **Result:** **30 rows** of noise data were removed. Remaining rows: 1866.

#### Aldi (`aldi_drinks`)

* **Handling Logic:** Data was sorted by price descendingly. An outlier was identified as a non-beverage machine.
* **Outlier Removed:** 1 row (Product: "Coffee Capsule Machine," Price: 78.99).
* **Result:** Remaining rows: 176.

#### Coles (`coles_drinks`)

* **Assessment:** No obvious noise data or outliers were detected.

### 3.1.3 Missing Value Imputation 

Initial filling of missing price data based on assumed promotional logic.

#### Woolworths (`wws_drinks`)

| Modification | Original State | Imputation Logic | Imputation Method | Rows Modified |
| :--- | :--- | :--- | :--- | :--- |
| **Online Price** | `online_price` is missing | If `online_price` is missing, assume it equals the original price. | `online_price = online_was_price` | 54 |
| **In-store Price** | `instore_price` is missing | If `instore_price` is missing, assume it equals the original price. | `instore_price = instore_was_price` | 54 |

#### Coles (`coles_drinks`)

| Modification | Target Object | Imputation Logic | Imputation Method | Rows Modified |
| :--- | :--- | :--- | :--- | :--- |
| **Modification 1** | `online_was_price` = 0 (Missing) | Products without an original price are assumed to be non-discounted. | `online_was_price = online_price` | 1060 |
| **Modification 2** | `save_percent` is blank AND `online_was_price` = 0 | Products with no original price and no save percent are not discounted. | `save_percent = 0` | 1060 |
| **Modification 3** | `save_percent` is blank AND `online_was_price` $\ne$ 0 | Calculate the discount percentage using known price data. | `save_percent = (online\_was\_price - online\_price) / online\_was\_price * 100` | 494 |

### 3.1.4 Data Standardization (Unit & Format)

#### Aldi (`aldi_drinks`)

* **Modification:** `instore_price`, `instore_was_price`.
* **Logic:** Original data was recorded in **cents**.
* **Method:** Multiply the entire column by $\mathbf{0.01}$ to convert to dollars.
* **Rows Modified:** 177

#### Coles (`coles_drinks`)

* **Modification:** `save_percent`.
* **Logic:** Convert the value to a percentage format and limit to 2 decimal places.
* **Method:** Multiply the entire column by $\mathbf{0.01}$ and apply percentage formatting.
* **Rows Modified:** 1484

## 3.2 Schema Standardization and Data Unification (Pandas)

This phase transitions the data processing to Python/Pandas to efficiently handle schema inconsistencies, column standardization, and the final data union.

### 3.2.1 Schema Standardization and Column Alignment

The three retail datasets (`wws_drinks`, `aldi_drinks`, `coles_drinks`) have differing column structures due to varying data availability from their respective websites. The goal of this step is to enforce a uniform schema across all tables prior to unification.

#### **Canonical Column List (`CANON_COLS`)**
CANON_COLS = [
    "retailer", "name", "brand", "package_size",
    "instore_price", "instore_was_price",
    "online_price", "online_was_price", "save_percent",
    "category_original", "category_std"
]

In [1]:
import pandas as pd
from pathlib import Path

XSLX = "drinks.xlsx"
xls = pd.ExcelFile(XSLX)

sheets = pd.read_excel(XSLX, sheet_name=None, dtype=str, keep_default_na=False)
retailer_map = {
    "Woolworth" : "wws_drinks",
    "Aldi" : "aldi_drinks",
    "Coles" : "coles_drinks"
}
CANON_COLS = [
    "retailer", "name", "brand", "package_size",
    "instore_price","instore_was_price",
    "online_price", "online_was_price", "save_percent",
    "category_original", "category_std"
]

def check_cols(df:pd.DataFrame, cols):
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df

def standardize(df_raw:pd.DataFrame, retailer:str):
    df = df_raw.copy()
    df["retailer"] = retailer
    df["category_original"] = df["category"]
    df = check_cols(df, CANON_COLS)
    df = df[CANON_COLS]
    return df

std_dfs = {}
for (retailer,sheet_name) in retailer_map.items():
    df_raw = sheets[sheet_name]
    df_std = standardize(df_raw, retailer)
    std_dfs[retailer] = df_std


out_dir = Path("intermediate")
out_dir.mkdir(exist_ok=True)
for retailer, df in std_dfs.items():
    df.to_csv(out_dir / f"{retailer.lower()}.csv", index=False)

#### Result
Three standardized intermediate datasets were saved: 'woolworth.csv', 'coles.csv','aldi.csv'
Each possessing an identical column structure and order,

### 3.2 Category Unification and Standardization (`category_std`)

The raw data presents a major challenge due to the lack of a consistent classification system across the three retailers, leading to high category complexity and inconsistency.

#### **Standardized Category List**

To enable comparative analysis, all products were mapped to the following eight unified categories:

In [7]:
CATEGORIES = [
    "Non/Low Alcoholic",
    "Water",
    "Milk",
    "Coffee",
    "Soft Drink",
    "Sport & Energy",
    "Tea",
    "Juice",
    "Other drinks",
]

#### **Category Standardization Logic: Two-Pass Regular Expression Approach**

Due to the extreme ambiguity and complex keyword concatenation used in the original `category` columns (e.g., Woolworths and Coles often combine keywords like `Tea & [Coffee,"Everyday Market","Coffee"]` or `[ENERGY/SPORT/ICEDTEA]`), direct mapping from the original category column was impractical.

The standardization was performed in a **two-pass iterative process** primarily relying on the **product name** (`name`) column.

##### **Pass 1: Name-Based Regular Expression Mapping**

* **Logic:** Products were initially classified by applying comprehensive regular expressions (`pat` variables) against the product **`name`** field.
* **Goal:** To capture the vast majority of clearly identifiable products (e.g., "Coca Cola" for "Soft Drink").
* **Result:** Ambiguously named products (e.g., "Variety Pack 30 Pack 375ml" or "Original Lemon 600ml") that could not be determined solely from the name were temporarily classified into **"Other drinks"**.

##### **Pass 2: Re-classification of "Other drinks" using Original Category**

* **Problem:** After Pass 1, a significant number of products remained in the "Other drinks" bucket (e.g., 340 items in Woolworths initially).
* **Logic:** The `category_original` column was then used as a secondary source. A simpler regular expression pass was executed *only* on the products classified as "Other drinks" to utilize keywords from the original, retailer-specific category data.
* **Result:** This secondary pass successfully re-classified approximately **90%** of the remaining "Other drinks" items. (Example: Woolworths' "Other drinks" count dropped to 11 items after this pass).

In [9]:
import warnings
from pandas.core.common import SettingWithCopyWarning 

warnings.filterwarnings(
    'ignore', 
    message="This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.",
    category=UserWarning
)
warnings.filterwarnings("ignore", category=SettingWithCopyWarning) 


FLAVORS = r"(apple|orange|mango|pineapple|lychee|peach|pear|grape|grapefruit|berry|strawberry|blueberry|raspberry|blackberry|cranberry|lemon|lime|passion(?:fruit)?|guava|pomegranate|watermelon|melon|kiwi|apricot|cherry|yuzu)"
# the order is seted by the consideration of:
# ginger beer (belong to soft drink instead of non Alcoholic)
# tonic water (soft drink / not water)
# Gatorade G Active Sports Drinks Berry Water Electrolytes （sport & energy/ not water)
# so, soft drink/sport & energy > water/ alcoholic
name_patterns = {
    "Milk": r"\b(milk|oat\s+milk|almond\s+milk|dairy)\b",
    "Sport & Energy": r"\b(sports?|electrolytes?|energy|enery|protein)\b",
    "Soft Drink": rf"(?:\b(?:tonic|ginger\s+(?:ale|beer)|cola|soft drink|lemonade|pepsi|sunkist|passiona|cream\s+soda)\b|(?:\b{FLAVORS}\b[\s-]*soda(?!\s*water)|\bsoda\b(?!\s*water)[\s-]*\b{FLAVORS}\b))",
    "Non/Low Alcoholic": r"\b(beer|vodka|alcohol|heineken|non[-\s]?alc|alcohol[-\s]?free)\b",
    "Coffee": r"\b(coffe(e)|iced\s*coffee|latte|cappuccino|mocha|espresso|americano)\b",
    "Tea": rf"\b(?:(?:tea|herbal|iced\s*tea|green\s*tea|black\s*tea|matcha|earl\s*grey|oolong|chai|k[ou]mbu?cha))\b",
    "Juice": rf"\b(?:(?:juice|nectar|fruit\s*drink|cordial|syrup|concentrate)|(?:.*?\b{FLAVORS}\b.*))\b",
    "Water": r"\b(water)\b"
}

category_patterns = {
    "Milk": r"\b(milk|oat\s+milk|almond\s+milk|dairy)\b",
    "Sport & Energy": r"\b(sports drinks|electrolytes?|energy|protein)\b",
    "Soft Drink": r"(soft\s?drinks?|cold\s?drink)",
    "Tea" : r"(tea)",
    "Juice" : r"(juice|coridials)",
    "Coffee" : r"coffee",
    "Non/Low Alcoholic": r"\b(beer|alcohol|wine|non[-\s]?alc|alcoholic)\b",
    "Water" : r"water",
}
def standardize_category(df: pd.DataFrame):
    out = df.copy()
    out["category_std"] = out["category_std"].fillna("Unmapped")

    base = out["category_std"].eq("Unmapped")
    # Firstly Use 'name' to decide category
    for cat, pat in name_patterns.items():
        m = base & out["name"].str.contains(pat, case=False, na=False, regex=True)
        out.loc[m, "category_std"] = cat
        base = out["category_std"].eq("Unmapped")

    # Use "category_original" decide if unmapped after name_patterns.
    for cat, pat in category_patterns.items():
        m = base & out["category_original"].str.contains(pat, case=False, na=False, regex=True)
        out.loc[m, "category_std"] = cat
        base = out["category_std"].eq("Unmapped")

    out.loc[out["category_std"].eq("Unmapped"), "category_std"] = "Other drinks"
    return out

inter_dir = Path("intermediate")
in_files = {
    "Woolworth": inter_dir / "woolworth.csv",
    "Aldi":      inter_dir / "aldi.csv",
    "Coles":     inter_dir / "coles.csv",
}

for retailer, path in in_files.items():
    df = pd.read_csv(path, dtype=str, keep_default_na=True)
    df2 = standardize_category(df)
    df2.to_csv(path, index=False)
    
    print(f"\n{retailer}:")
    print(df2["category_std"].value_counts(dropna=False))
    



Woolworth:
Juice                442
Coffee               391
Tea                  300
Soft Drink           295
Sport & Energy       158
Milk                 156
Water                 70
Non/Low Alcoholic     43
Other drinks          11
Name: category_std, dtype: int64

Aldi:
Coffee            51
Juice             42
Soft Drink        34
Tea               21
Water             11
Milk               9
Sport & Energy     8
Name: category_std, dtype: int64

Coles:
Juice                375
Coffee               337
Soft Drink           238
Tea                  219
Milk                 126
Sport & Energy       118
Water                 55
Non/Low Alcoholic     15
Other drinks           1
Name: category_std, dtype: int64


#### **Final Outcome**

* **Final `Other drinks`:** This category now serves as the final **catch-all bucket** for truly ambiguous or unique products that cannot be reliably classified (e.g., "Remedy Shots Digestion 12 x 60ml").
* **Conclusion:** Following this two-pass iterative adjustment, the category schemas of all three retailers were unified, and the classification accuracy was judged to be near-precise after manual verification.

### 3.3 Price Granularity Alignment and Imputation 
This step focuses on addressing the retailer-specific gaps in pricing columns (`save_percent`, `instore_price`, `instore_was_price`) to ensure a consistent data granularity across the three finalized tables before the final union.

#### Woolworths: Imputing Missing Discount Depth (`save_percent`)

Woolworths data lacked a pre-calculated `save_percent` column. This was calculated using the available *in-store* price fields.

* **Logic:** For records where `save_percent` was missing (`NaN`), the value was calculated using the formula: $$\text{Save Percent} = \frac{(\text{instore_was_price} - \text{instore_price})}{\text{instore_was_price}}$$

In [10]:
import numpy as np

inter_dir = Path('intermediate')
wws = inter_dir/"woolworth.csv"
df_wws = pd.read_csv(wws)

df_wws['instore_was_price'] = pd.to_numeric(df_wws['instore_was_price'])
df_wws['instore_price'] = pd.to_numeric(df_wws['instore_price'])

save_percent = (df_wws['instore_was_price'] - df_wws['instore_price']) / df_wws['instore_was_price']
df_wws['save_percent'] = df_wws['save_percent'].fillna(save_percent)

df_wws.to_csv(wws, index=False)

#### Aldi: Imputing Missing Discount Depth (`save_percent`)
Same as Woolworth

In [11]:
aldi = inter_dir/"aldi.csv"
df_aldi = pd.read_csv(aldi)

df_aldi['instore_was_price'] = pd.to_numeric(df_aldi['instore_was_price'])
df_aldi['instore_price'] = pd.to_numeric(df_aldi['instore_price'])

save_percent = (df_aldi['instore_was_price'] - df_aldi['instore_price']) / df_aldi['instore_was_price']
df_aldi['save_percent'] = df_aldi['save_percent'].fillna(save_percent)

df_aldi.to_csv(aldi, index=False)

#### Coles: Aligning Online and In-store Prices
Coles data only provides online pricing. To align the data with the other retailers, Coles' online prices were explicitly copied to the in-store price fields to fill the missing instore columns.

In [12]:
coles = inter_dir/'coles.csv'
df_coles = pd.read_csv(coles)

df_coles['instore_price'] = df_coles['instore_price'].fillna(df_coles['online_price'])
df_coles['instore_was_price'] = df_coles['instore_was_price'].fillna(df_coles['online_was_price'])

df_coles.to_csv(coles, index=False)

### 3.4 Union Tables 

In [13]:
union = "union_drinks.csv"
files = [inter_dir/"woolworth.csv", inter_dir/"aldi.csv", inter_dir/"coles.csv"]

df_list = []
for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

df_union = pd.concat(df_list, ignore_index=True)
print(f"Total: {len(df_union)}")

df_union.to_csv(union, index=False, encoding='utf-8')
print("Done")

Total: 3526
Done


## 4. Conclusion and Final Output Summary

This comprehensive data cleaning and unification process successfully transformed the raw, disparate datasets into a single, standardized analytical table.

### Final Dataset Structure

The final output is the unified file **`union_drinks.csv`**, which contains a total of **3,526 records**, structured by the 11 canonical fields defined in the schema standardization phase.

| Supermarket | Final Record Count |
| :--- | :--- |
| Woolworths | 1,866 |
| Aldi | 176 |
| Coles | 1,484 |
| **TOTAL** | **3,526** |

### Canonical Fields 

Each record in the final dataset includes the following 11 fields, with consistent definitions and data types enforced:

| Field | Data Type (Pandas/SQL) | Description |
| :--- | :--- | :--- |
| **1. retailer** | `str` (Object) | The source supermarket (Woolworths, Aldi, Coles). |
| **2. name** | `str` (Object) | Product display name. |
| **3. brand** | `str` (Object) | Product brand name. |
| **4. package_size** | `str` (Object) | Packaging volume or size. |
| **5. instore_price** | `double` (float64) | Current price |
| **6. instore_was_price** | `double` (float64) | Original/RRP price |
| **7. online_price** | `double` (float64) | Current online price |
| **8. online_was_price** | `double` (float64) | Original online price |
| **9. save_percent** | `double` (float64) | Calculated discount depth |
| **10. category_original** | `str` (Object) | The original category classification from the source website. |
| **11. category_std** | `str` (Object) | The final, unified category (8 standard types). |

### 5.3 Key Achievements

* **Schema Consistency:** All three sources are now aligned to a single, identical column schema.
* **Price Granularity:** In-store and online pricing fields were aligned and imputed (where necessary) across all retailers (e.g., Coles online prices were used for instore alignment; Aldi's instore prices were used for online alignment).
* **Unified Categories:** The complex, retailer-specific category system has been successfully mapped to a unified set of 8 standard beverage categories, ensuring comparability for analysis.

The resulting dataset is now clean, integrated, and ready for advanced statistical and comparative analysis.

In [14]:
out = "union_drinks.xlsx"
with pd.ExcelWriter(out, engine='xlsxwriter') as writer:
    df_union.to_excel(writer, index=False)